# Part 1: Input Preprocessing

## 1.a Intent Classification

In [2]:
%pip install openai python-dotenv neo4j -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from typing import Dict, List, Any, Optional
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from neo4j import GraphDatabase

load_dotenv()

True

### 5 Intent Categories

| Intent | Purpose | Example |
|--------|---------|----------|
| LIST_HOTELS | Find multiple hotels | "Show hotels in Paris" |
| RECOMMEND_HOTEL | Get personalized suggestions | "Recommend a hotel for families" |
| DESCRIBE_HOTEL | Get details about one hotel | "Tell me about Hilton Cairo" |
| COMPARE_HOTELS | Compare multiple hotels | "Compare Hilton vs Marriott" |
| CHECK_VISA | Visa requirements | "Do I need a visa for Turkey?" |

In [4]:
class IntentClassifier:
    
    def __init__(self):
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.model = "gpt-4o-mini"
        self.intents = {
            "LIST_HOTELS": "Find multiple hotels matching filters",
            "RECOMMEND_HOTEL": "Get personalized suggestions",
            "DESCRIBE_HOTEL": "Get details about one hotel",
            "COMPARE_HOTELS": "Compare multiple hotels",
            "CHECK_VISA": "Check visa requirements"
        }
    
    def classify(self, user_query: str) -> Optional[str]:
        prompt = f"""Classify this query into ONE intent: {list(self.intents.keys())} or return NONE.
        
        Intent definitions:
        - LIST_HOTELS: neutral search (keywords: show, find, list)
        - RECOMMEND_HOTEL: opinions/advice (keywords: recommend, suggest, best, top)
        - DESCRIBE_HOTEL: one specific hotel (must mention hotel name)
        - COMPARE_HOTELS: multiple hotels (keywords: compare, vs, which is better)
        - CHECK_VISA: visa requirements (keywords: visa, entry requirement)
        
        Query: \"{user_query}\"
        
        Return ONLY the intent name or NONE."""
        
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.0,
                max_tokens=20
            )
            intent = response.choices[0].message.content.strip().upper()
            return intent if intent in self.intents else None
        except Exception as e:
            print(f"Error: {e}")
            return None

classifier = IntentClassifier()
print("IntentClassifier initialized")

IntentClassifier initialized


In [5]:
test_intents = [
    ("Show me hotels in Paris", "LIST_HOTELS"),
    ("Recommend a hotel for families in Dubai", "RECOMMEND_HOTEL"),
    ("Tell me about Hilton Cairo", "DESCRIBE_HOTEL"),
    ("Compare Hilton and Marriott", "COMPARE_HOTELS"),
    ("Do I need a visa for Turkey?", "CHECK_VISA"),
    ("What's the weather?", None),
]

correct = 0
for query, expected in test_intents:
    result = classifier.classify(query)
    status = "PASS" if result == expected else "FAIL"
    if result == expected:
        correct += 1
    print(f"[{status}] '{query}' -> {result}")

print(f"\nIntent Classification: {correct}/{len(test_intents)} passed")

[PASS] 'Show me hotels in Paris' -> LIST_HOTELS
[PASS] 'Recommend a hotel for families in Dubai' -> RECOMMEND_HOTEL
[PASS] 'Tell me about Hilton Cairo' -> DESCRIBE_HOTEL
[PASS] 'Compare Hilton and Marriott' -> COMPARE_HOTELS
[PASS] 'Do I need a visa for Turkey?' -> CHECK_VISA
[PASS] 'What's the weather?' -> None

Intent Classification: 6/6 passed


## 1.b Entity Extraction

### Entity Schemas by Intent

| Intent | Entities |
|--------|----------|
| LIST_HOTELS | city, country, star_rating |
| RECOMMEND_HOTEL | city, country, traveller_type, age_group, user_gender, star_rating, aspects |
| DESCRIBE_HOTEL | hotel_name, aspects |
| COMPARE_HOTELS | hotel1, hotel2, traveller_type, aspects |
| CHECK_VISA | from_country, to_country |

In [6]:
SCHEMAS: Dict[str, Dict[str, Any]] = {
    "LIST_HOTELS": {"city": None, "country": None, "star_rating": None},
    "RECOMMEND_HOTEL": {"city": None, "country": None, "traveller_type": None, 
                        "age_group": None, "user_gender": None, "star_rating": None, "aspects": None},
    "DESCRIBE_HOTEL": {"hotel_name": None, "aspects": None},
    "COMPARE_HOTELS": {"hotel1": None, "hotel2": None, "traveller_type": None, "aspects": None},
    "CHECK_VISA": {"from_country": None, "to_country": None}
}

ALLOWED_ASPECTS = ["cleanliness", "comfort", "facilities", "location", "staff", "value_for_money"]

print(f"Schemas: {list(SCHEMAS.keys())}")
print(f"Allowed aspects: {ALLOWED_ASPECTS}")

Schemas: ['LIST_HOTELS', 'RECOMMEND_HOTEL', 'DESCRIBE_HOTEL', 'COMPARE_HOTELS', 'CHECK_VISA']
Allowed aspects: ['cleanliness', 'comfort', 'facilities', 'location', 'staff', 'value_for_money']


In [7]:
def enforce_schema(intent: str, entities: Dict[str, Any]) -> Dict[str, Any]:
    schema = SCHEMAS[intent]
    result = {}
    
    for key in schema.keys():
        value = entities.get(key, None)
        
        if key == "star_rating" and value is not None:
            try:
                value = int(value)
                if not (1 <= value <= 5):
                    value = None
            except (ValueError, TypeError):
                value = None
        
        elif key == "aspects" and value is not None:
            if isinstance(value, str):
                value = [value]
            if isinstance(value, list):
                normalized = []
                for asp in value:
                    if isinstance(asp, str):
                        asp_clean = asp.lower().strip().replace(" ", "_").replace("-", "_")
                        if asp_clean in ALLOWED_ASPECTS:
                            normalized.append(asp_clean)
                value = list(set(normalized)) if normalized else None
            else:
                value = None
        
        elif key == "traveller_type" and value is not None:
            if isinstance(value, str):
                value = value.lower().strip()
                if value not in ["family", "solo", "couple", "business", "group"]:
                    value = None
        
        elif key == "user_gender" and value is not None:
            if isinstance(value, str):
                value = value.lower().strip()
                if value not in ["male", "female"]:
                    value = None
        
        result[key] = value
    
    return result

print("Schema enforcement function defined")

Schema enforcement function defined


In [8]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def extract_entities(text: str, intent: str) -> Dict[str, Any]:
    if intent not in SCHEMAS:
        return dict(SCHEMAS.get("LIST_HOTELS", {}))
    
    prompt = f"""Extract entities from this query and return ONLY valid JSON.
    
    Query: \"{text}\"
    Intent: {intent}
    Required keys: {list(SCHEMAS[intent].keys())}
    
    RULES:
    1. Extract ONLY explicitly mentioned entities
    2. Vague words (good, best, nice) do NOT extract aspects
    3. Aspects only if explicitly mentioned (e.g., "clean rooms" -> cleanliness)
    4. For possessive forms (e.g., "hotel's cleanliness"), extract the aspect after the possessive
    5. Preserve complete hotel names including articles (e.g., "The Azure Tower", not "Azure Tower")
    6. Allowed aspects: {ALLOWED_ASPECTS}
    7. traveller_type: family, solo, couple, business, group
    8. user_gender: male, female
    9. star_rating: 1-5 (numeric)
    
    Return ONLY JSON matching: {SCHEMAS[intent]}"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,
            max_tokens=200
        )
        
        raw = response.choices[0].message.content.strip()
        if raw.startswith("```"):
            raw = raw.split("```")[1]
            if raw.startswith("json"):
                raw = raw[4:]
            raw = raw.strip()
        
        entities = json.loads(raw)
        return enforce_schema(intent, entities)
    except Exception as e:
        print(f"Error: {e}")
        return dict(SCHEMAS[intent])

print("Entity extraction function defined")

Entity extraction function defined


In [9]:
test_cases = [
    ("Recommend a hotel for families in Dubai with clean rooms", "RECOMMEND_HOTEL"),
    ("I want good hotels for family in Paris", "RECOMMEND_HOTEL"),
    ("Compare Hilton Cairo and Marriott Cairo", "COMPARE_HOTELS"),
    ("Describe Hilton Dubai", "DESCRIBE_HOTEL"),
    ("Do Egyptians need a visa for Turkey?", "CHECK_VISA"),
]

for query, intent in test_cases:
    result = extract_entities(query, intent)
    print(f"\nQuery: {query}")
    print(f"Intent: {intent}")
    print(f"Extracted: {json.dumps({k: v for k, v in result.items() if v is not None}, indent=2)}")


Query: Recommend a hotel for families in Dubai with clean rooms
Intent: RECOMMEND_HOTEL
Extracted: {
  "city": "Dubai",
  "traveller_type": "family",
  "aspects": [
    "cleanliness"
  ]
}

Query: I want good hotels for family in Paris
Intent: RECOMMEND_HOTEL
Extracted: {
  "city": "Paris",
  "traveller_type": "family"
}

Query: Compare Hilton Cairo and Marriott Cairo
Intent: COMPARE_HOTELS
Extracted: {
  "hotel1": "Hilton Cairo",
  "hotel2": "Marriott Cairo"
}

Query: Describe Hilton Dubai
Intent: DESCRIBE_HOTEL
Extracted: {
  "hotel_name": "Hilton Dubai"
}

Query: Do Egyptians need a visa for Turkey?
Intent: CHECK_VISA
Extracted: {
  "from_country": "Egypt",
  "to_country": "Turkey"
}


## 1.c Input Embedding

In [10]:
%pip install sentence-transformers -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Primary embedding model loaded: all-MiniLM-L6-v2")


Primary embedding model loaded: all-MiniLM-L6-v2


In [12]:
def embed_query(query: str):
    embedding = embedder.encode([query], convert_to_numpy=True)
    return embedding[0]

print("Query embedding function defined")

Query embedding function defined


# Part 2: Graph Retrieval

## 2.a Baseline (Cypher Queries)

In [13]:
class Neo4jConnection:
    def __init__(self, config_path=None):
        if config_path is None:
            possible_paths = [
                os.path.join('KnowledgeGraph', 'config.txt'),
                os.path.join('Milestone 3', 'KnowledgeGraph', 'config.txt'),
            ]
            config_path = None
            for path in possible_paths:
                if os.path.exists(path):
                    config_path = path
                    break
        
        config = {}
        with open(config_path, 'r') as f:
            for line in f:
                if '=' in line:
                    key, value = line.strip().split('=', 1)
                    config[key] = value
        
        self.driver = GraphDatabase.driver(config['URI'], auth=(config['USERNAME'], config['PASSWORD']))
    
    def execute_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters or {})
            return [dict(record) for record in result]
    
    def close(self):
        self.driver.close()

print("Neo4jConnection class defined")

Neo4jConnection class defined


### 14 Query Templates by Intent

- **LIST_HOTELS**: L1-L5 (5 variants)
- **RECOMMEND_HOTEL**: R1, R3-R5 (4 variants, R2 removed)
- **DESCRIBE_HOTEL**: D1-D2 (2 variants)
- **COMPARE_HOTELS**: C1-C2 (2 variants)
- **CHECK_VISA**: V1 (1 variant)

In [14]:
class QueryLibrary:
    
    @staticmethod
    def template_L1_list_by_city(conn: Neo4jConnection, city: str):
        query = """MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, h.star_rating AS star_rating
        ORDER BY h.star_rating DESC LIMIT 50"""
        return conn.execute_query(query, {'city': city})
    
    @staticmethod
    def template_L2_list_by_country(conn: Neo4jConnection, country: str):
        query = """MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE country.name = $country RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, h.star_rating AS star_rating
        ORDER BY h.star_rating DESC LIMIT 50"""
        return conn.execute_query(query, {'country': country})
    
    @staticmethod
    def template_L3_list_by_rating(conn: Neo4jConnection, star_rating: int):
        query = """MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE h.star_rating = $star_rating RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, h.star_rating AS star_rating
        ORDER BY h.name LIMIT 50"""
        return conn.execute_query(query, {'star_rating': star_rating})
    
    @staticmethod
    def template_L4_list_by_city_and_rating(conn: Neo4jConnection, city: str, star_rating: int):
        query = """MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city AND h.star_rating = $star_rating 
        RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, h.star_rating AS star_rating LIMIT 50"""
        return conn.execute_query(query, {'city': city, 'star_rating': star_rating})
    
    @staticmethod
    def template_L5_list_by_country_and_rating(conn: Neo4jConnection, country: str, star_rating: int):
        query = """MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE country.name = $country AND h.star_rating = $star_rating
        RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, h.star_rating AS star_rating LIMIT 50"""
        return conn.execute_query(query, {'country': country, 'star_rating': star_rating})
    
    @staticmethod
    def template_R1_recommend_by_location(conn: Neo4jConnection, city: str, star_rating: int = None):
        where_parts = ["city.name = $city"]
        params = {'city': city}
        if star_rating:
            where_parts.append("h.star_rating = $star_rating")
            params['star_rating'] = star_rating
        where_clause = " AND ".join(where_parts)
        
        query = f"""MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE {where_clause} OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)
        WITH h, city, country, collect(r) AS reviews WHERE size(reviews) > 0 UNWIND reviews AS r
        RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, avg(r.score_overall) AS overall_review_score
        ORDER BY overall_review_score DESC LIMIT 10"""
        return conn.execute_query(query, params)
    
    @staticmethod
    def template_R3_recommend_by_aspects(conn: Neo4jConnection, city: str, aspects: List[str], 
                                        age_group=None, user_gender=None, star_rating: int = None):
        aspect_mapping = {'cleanliness': 'score_cleanliness', 'comfort': 'score_comfort', 'facilities': 'score_facilities',
                         'location': 'score_location', 'staff': 'score_staff', 'value_for_money': 'score_value_for_money'}
        
        valid_aspects = [a for a in (aspects or []) if a in aspect_mapping]
        if not valid_aspects:
            return []
        
        aspect_avg = " + ".join([f"coalesce(avg(r.{aspect_mapping[a]}), 0)" for a in valid_aspects])
        aspect_select = ", ".join([f"avg(r.{aspect_mapping[a]}) AS {a}_review" for a in valid_aspects])
        
        where_parts = ["city.name = $city"]
        params = {'city': city}
        if star_rating:
            where_parts.append("h.star_rating = $star_rating")
            params['star_rating'] = star_rating
        
        demo_conditions = []
        if age_group:
            demo_conditions.append("u.age_group = $age_group")
            params['age_group'] = age_group
        if user_gender:
            demo_conditions.append("u.gender = $user_gender")
            params['user_gender'] = user_gender
        
        where_clause = " AND ".join(where_parts)
        user_match = "<-[:WROTE]-(u:User)" if demo_conditions else ""
        demo_clause = " AND " + " AND ".join(demo_conditions) if demo_conditions else ""
        
        query = f"""MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE {where_clause} OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review){user_match} WHERE TRUE{demo_clause}
        WITH h, city, country, collect(r) AS reviews WHERE size(reviews) > 0 UNWIND reviews AS r
        RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, {aspect_select},
        ({aspect_avg}) / {len(valid_aspects)} AS composite_aspect_score, count(r) AS review_count
        ORDER BY composite_aspect_score DESC LIMIT 10"""
        
        return conn.execute_query(query, params)
    
    @staticmethod
    def template_R4_recommend_by_traveller_and_aspects(conn: Neo4jConnection, city: str, traveller_type: str, 
                                                       aspects: List[str], age_group=None, user_gender=None, star_rating: int = None):
        aspect_mapping = {'cleanliness': 'score_cleanliness', 'comfort': 'score_comfort', 'facilities': 'score_facilities',
                         'location': 'score_location', 'staff': 'score_staff', 'value_for_money': 'score_value_for_money'}
        
        valid_aspects = [a for a in (aspects or []) if a in aspect_mapping]
        if not valid_aspects:
            return QueryLibrary.template_R3_recommend_by_aspects(conn, city, aspects, age_group, user_gender, star_rating)
        
        aspect_avg = " + ".join([f"coalesce(avg(r.{aspect_mapping[a]}), 0)" for a in valid_aspects])
        aspect_select = ", ".join([f"avg(r.{aspect_mapping[a]}) AS {a}_review" for a in valid_aspects])
        
        where_parts = ["city.name = $city"]
        params = {'city': city, 'traveller_type': traveller_type}
        if star_rating:
            where_parts.append("h.star_rating = $star_rating")
            params['star_rating'] = star_rating
        where_clause = " AND ".join(where_parts)
        
        conditions = ["t.type = $traveller_type"]
        if age_group:
            conditions.append("u.age_group = $age_group")
            params['age_group'] = age_group
        if user_gender:
            conditions.append("u.gender = $user_gender")
            params['user_gender'] = user_gender
        
        traveller_where = " AND ".join(conditions)
        user_match = "<-[:WROTE]-(u:User)" if (age_group or user_gender) else ""
        
        query = f"""MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE {where_clause} OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)<-[:WROTE]-(t:Traveller){user_match}
        WHERE {traveller_where} WITH h, city, country, collect(r) AS reviews WHERE size(reviews) > 0 UNWIND reviews AS r
        RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, {aspect_select},
        ({aspect_avg}) / {len(valid_aspects)} AS composite_aspect_score, count(r) AS review_count
        ORDER BY composite_aspect_score DESC LIMIT 10"""
        
        results = conn.execute_query(query, params)
        if not results:
            results = QueryLibrary.template_R3_recommend_by_aspects(conn, city, aspects, age_group, user_gender, star_rating)
        return results
    
    @staticmethod
    def template_R5_recommend_with_rating_filter(conn: Neo4jConnection, city: str, star_rating: int):
        query = """MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city AND h.star_rating = $star_rating OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)
        WITH h, city, country, collect(r) AS reviews WHERE size(reviews) > 0 UNWIND reviews AS r
        RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, avg(r.score_overall) AS overall_review_score
        ORDER BY overall_review_score DESC LIMIT 10"""
        return conn.execute_query(query, {'city': city, 'star_rating': star_rating})
    
    @staticmethod
    def template_D1_describe_all_aspects(conn: Neo4jConnection, hotel_name: str):
        query = """MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE toLower(h.name) = toLower($hotel_name) OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)
        RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name,
        h.cleanliness_base AS cleanliness_base, h.comfort_base AS comfort_base, h.facilities_base AS facilities_base, 
        h.location_base AS location_base, h.staff_base AS staff_base, h.value_for_money_base AS value_for_money_base,
        avg(r.score_cleanliness) AS cleanliness_review, avg(r.score_comfort) AS comfort_review,
        avg(r.score_facilities) AS facilities_review, avg(r.score_location) AS location_review,
        avg(r.score_staff) AS staff_review, avg(r.score_value_for_money) AS value_for_money_review, 
        count(r) AS review_count LIMIT 1"""
        return conn.execute_query(query, {'hotel_name': hotel_name})
    
    @staticmethod
    def template_D2_describe_specific_aspects(conn: Neo4jConnection, hotel_name: str, aspects: List[str]):
        aspect_mapping = {'cleanliness': ('cleanliness_base', 'score_cleanliness'), 'comfort': ('comfort_base', 'score_comfort'),
                         'facilities': ('facilities_base', 'score_facilities'), 'location': ('location_base', 'score_location'),
                         'staff': ('staff_base', 'score_staff'), 'value_for_money': ('value_for_money_base', 'score_value_for_money')}
        
        valid_aspects = [a for a in aspects if a in aspect_mapping]
        if not valid_aspects:
            return QueryLibrary.template_D1_describe_all_aspects(conn, hotel_name)
        
        aspect_fields = []
        for aspect in valid_aspects:
            base_field, review_field = aspect_mapping[aspect]
            aspect_fields.append(f"h.{base_field} AS {aspect}_base")
            aspect_fields.append(f"avg(r.{review_field}) AS {aspect}_review")
        
        aspect_select = ", ".join(aspect_fields)
        query = f"""MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE toLower(h.name) = toLower($hotel_name) OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)
        RETURN h.name AS hotel_name, city.name AS city_name, country.name AS country_name, {aspect_select}, count(r) AS review_count LIMIT 1"""
        return conn.execute_query(query, {'hotel_name': hotel_name})
    
    @staticmethod
    def template_C1_compare_all_aspects(conn: Neo4jConnection, hotel1: str, hotel2: str, aspects: List[str] = None):
        aspect_mapping = {'cleanliness': 'cleanliness_base', 'comfort': 'comfort_base', 'facilities': 'facilities_base',
                         'location': 'location_base', 'staff': 'staff_base', 'value_for_money': 'value_for_money_base'}
        
        if aspects:
            valid_aspects = [a for a in aspects if a in aspect_mapping]
            if not valid_aspects:
                return []
        else:
            valid_aspects = list(aspect_mapping.keys())
        
        aspect_fields = []
        for aspect in valid_aspects:
            base_field = aspect_mapping[aspect]
            aspect_fields.append(f"h1.{base_field} AS hotel1_{aspect}_base")
            aspect_fields.append(f"h2.{base_field} AS hotel2_{aspect}_base")
        
        aspect_select = ", ".join(aspect_fields)
        
        query = f"""MATCH (h1:Hotel)-[:LOCATED_IN]->(city1:City)-[:LOCATED_IN]->(country1:Country),
        (h2:Hotel)-[:LOCATED_IN]->(city2:City)-[:LOCATED_IN]->(country2:Country)
        WHERE toLower(h1.name) = toLower($hotel1) AND toLower(h2.name) = toLower($hotel2)
        RETURN h1.name AS hotel1_name, city1.name AS hotel1_city, country1.name AS hotel1_country,
        h2.name AS hotel2_name, city2.name AS hotel2_city, country2.name AS hotel2_country,
        {aspect_select} LIMIT 1"""
        return conn.execute_query(query, {'hotel1': hotel1, 'hotel2': hotel2})
    
    @staticmethod
    def template_C2_compare_with_traveller_type(conn: Neo4jConnection, hotel1: str, hotel2: str, 
                                               traveller_type: str, aspects: List[str] = None):
        aspect_mapping = {'cleanliness': 'cleanliness_base', 'comfort': 'comfort_base', 'facilities': 'facilities_base',
                         'location': 'location_base', 'staff': 'staff_base', 'value_for_money': 'value_for_money_base'}
        
        if aspects:
            valid_aspects = [a for a in aspects if a in aspect_mapping]
            if not valid_aspects:
                return QueryLibrary.template_C1_compare_all_aspects(conn, hotel1, hotel2)
        else:
            valid_aspects = list(aspect_mapping.keys())
        
        aspect_fields = []
        for aspect in valid_aspects:
            base_field = aspect_mapping[aspect]
            aspect_fields.append(f"h1.{base_field} AS hotel1_{aspect}_base")
            aspect_fields.append(f"h2.{base_field} AS hotel2_{aspect}_base")
        
        aspect_select = ", ".join(aspect_fields)
        
        query = f"""MATCH (h1:Hotel)-[:LOCATED_IN]->(city1:City)-[:LOCATED_IN]->(country1:Country),
        (h2:Hotel)-[:LOCATED_IN]->(city2:City)-[:LOCATED_IN]->(country2:Country)
        WHERE toLower(h1.name) = toLower($hotel1) AND toLower(h2.name) = toLower($hotel2)
        RETURN h1.name AS hotel1_name, city1.name AS hotel1_city, country1.name AS hotel1_country,
        h2.name AS hotel2_name, city2.name AS hotel2_city, country2.name AS hotel2_country,
        {aspect_select} LIMIT 1"""
        
        results = conn.execute_query(query, {'hotel1': hotel1, 'hotel2': hotel2, 'traveller_type': traveller_type})
        if not results:
            results = QueryLibrary.template_C1_compare_all_aspects(conn, hotel1, hotel2, aspects)
        return results
    
    @staticmethod
    def template_V1_check_visa_requirement(conn: Neo4jConnection, from_country: str, to_country: str):
        query = """MATCH (from:Country {name: $from_country}), (to:Country {name: $to_country})
        OPTIONAL MATCH (from)-[v:NEEDS_VISA]->(to)
        RETURN from.name AS from_country, to.name AS to_country, v.visa_type AS visa_type, 
        CASE WHEN v IS NOT NULL THEN true ELSE false END AS visa_required LIMIT 1"""
        return conn.execute_query(query, {'from_country': from_country, 'to_country': to_country})

print("QueryLibrary defined (14 templates)")

QueryLibrary defined (14 templates)


In [15]:
def select_and_execute_query(conn: Neo4jConnection, intent: str, entities: Dict[str, Any]):
    if intent == "LIST_HOTELS":
        city, country, star_rating = entities.get('city'), entities.get('country'), entities.get('star_rating')
        if city and star_rating:
            return QueryLibrary.template_L4_list_by_city_and_rating(conn, city, star_rating)
        elif country and star_rating:
            return QueryLibrary.template_L5_list_by_country_and_rating(conn, country, star_rating)
        elif city:
            return QueryLibrary.template_L1_list_by_city(conn, city)
        elif country:
            return QueryLibrary.template_L2_list_by_country(conn, country)
        elif star_rating:
            return QueryLibrary.template_L3_list_by_rating(conn, star_rating)
    
    elif intent == "RECOMMEND_HOTEL":
        city = entities.get('city')
        traveller_type = entities.get('traveller_type')
        aspects = entities.get('aspects')
        star_rating = entities.get('star_rating')
        age_group = entities.get('age_group')
        user_gender = entities.get('user_gender')
        
        if city:
            if traveller_type and aspects:
                return QueryLibrary.template_R4_recommend_by_traveller_and_aspects(
                    conn, city, traveller_type, aspects, age_group, user_gender, star_rating)
            elif aspects:
                return QueryLibrary.template_R3_recommend_by_aspects(
                    conn, city, aspects, age_group, user_gender, star_rating)
            elif star_rating and traveller_type:
                return QueryLibrary.template_R1_recommend_by_location(conn, city, star_rating)
            elif star_rating:
                return QueryLibrary.template_R5_recommend_with_rating_filter(conn, city, star_rating)
            elif traveller_type:
                return QueryLibrary.template_R1_recommend_by_location(conn, city)
            else:
                return QueryLibrary.template_R1_recommend_by_location(conn, city)
    
    elif intent == "DESCRIBE_HOTEL":
        hotel_name, aspects = entities.get('hotel_name'), entities.get('aspects')
        if hotel_name:
            return QueryLibrary.template_D2_describe_specific_aspects(conn, hotel_name, aspects) if aspects else QueryLibrary.template_D1_describe_all_aspects(conn, hotel_name)
    
    elif intent == "COMPARE_HOTELS":
        hotel1, hotel2, traveller_type, aspects = entities.get('hotel1'), entities.get('hotel2'), entities.get('traveller_type'), entities.get('aspects')
        if hotel1 and hotel2:
            return QueryLibrary.template_C2_compare_with_traveller_type(conn, hotel1, hotel2, traveller_type, aspects) if traveller_type else QueryLibrary.template_C1_compare_all_aspects(conn, hotel1, hotel2, aspects)
    
    elif intent == "CHECK_VISA":
        from_country, to_country = entities.get('from_country'), entities.get('to_country')
        if from_country and to_country:
            return QueryLibrary.template_V1_check_visa_requirement(conn, from_country, to_country)
    
    return []

print("Query selector defined")

Query selector defined


In [16]:
def full_pipeline(user_query: str, conn: Neo4jConnection):
    print(f"\n{'='*80}")
    print(f"User Query: {user_query}")
    print(f"{'='*80}")
    
    intent = classifier.classify(user_query)
    print(f"Intent: {intent}")
    
    if intent is None or intent == "NONE":
        print("No valid intent detected")
        return None
    
    entities = extract_entities(user_query, intent)
    entities_display = {k: v for k, v in entities.items() if v is not None}
    print(f"Extracted Entities: {json.dumps(entities_display, indent=2)}")
    
    results = select_and_execute_query(conn, intent, entities)
    print(f"Query Results: {len(results)} rows")
    if results and len(results) > 0:
        print(f"Sample: {json.dumps(results[0], indent=2)}")
    print()
    
    return results

try:
    conn = Neo4jConnection()
    
    test_queries = [
        ("L1", "Show hotels in Paris"),
        ("L2", "List hotels in France"),
        ("L3", "Show me 5-star hotels"),
        ("L4", "Show 5-star hotels in Cairo"),
        ("L5", "Show 5-star hotels in Egypt"),
        ("R1", "Recommend hotels in Cairo"),
        ("R3", "Recommend hotels in Cairo with good cleanliness"),
        ("R4", "Recommend hotels in Cairo for families with comfortable rooms"),
        ("R5", "Recommend 4-star hotels in Cairo"),
        ("D1", "Tell me about The Azure Tower"),
        ("D2", "Tell me about The Azure Tower's cleanliness"),
        ("C1", "Compare The Azure Tower and Nile Grandeur"),
        ("C2", "Compare The Azure Tower and Nile Grandeur for families"),
        ("V1", "Do Egyptians need a visa for Turkey?"),
    ]
    
    results_summary = []
    for template_id, query in test_queries:
        result = full_pipeline(query, conn)
        status = "PASS" if result else "EMPTY"
        results_summary.append((template_id, status))
    
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    passed = sum(1 for _, status in results_summary if status == "PASS")
    for template_id, status in results_summary:
        print(f"{template_id}: {status}")
    print(f"\nResult: {passed}/{len(test_queries)} templates executed successfully")
    
    conn.close()
except Exception as e:
    print(f"Error: {e}")


User Query: Show hotels in Paris
Intent: LIST_HOTELS
Extracted Entities: {
  "city": "Paris"
}
Query Results: 1 rows
Sample: {
  "hotel_name": "L'\u00c9toile Palace",
  "city_name": "Paris",
  "country_name": "France",
  "star_rating": 5.0
}


User Query: List hotels in France
Intent: LIST_HOTELS
Extracted Entities: {
  "country": "France"
}
Query Results: 1 rows
Sample: {
  "hotel_name": "L'\u00c9toile Palace",
  "city_name": "Paris",
  "country_name": "France",
  "star_rating": 5.0
}


User Query: Show me 5-star hotels
Intent: LIST_HOTELS
Extracted Entities: {
  "star_rating": 5
}
Query Results: 25 rows
Sample: {
  "hotel_name": "Aztec Heights",
  "city_name": "Mexico City",
  "country_name": "Mexico",
  "star_rating": 5.0
}


User Query: Show 5-star hotels in Cairo
Intent: LIST_HOTELS
Extracted Entities: {
  "city": "Cairo",
  "star_rating": 5
}
Query Results: 1 rows
Sample: {
  "hotel_name": "Nile Grandeur",
  "city_name": "Cairo",
  "country_name": "Egypt",
  "star_rating": 5.0
}

## 2.b Embeddings-Based Retrieval (RAG)

### Step 1: Extract Hotel + Review Data from Neo4j

In [17]:
conn_rag = Neo4jConnection()

query = """
MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)<-[:WROTE]-(t:Traveller)
WITH h, city, country, t.type AS traveller_type,
     avg(r.score_overall) AS avg_overall,
     avg(r.score_cleanliness) AS avg_cleanliness,
     avg(r.score_comfort) AS avg_comfort,
     avg(r.score_facilities) AS avg_facilities,
     avg(r.score_location) AS avg_location,
     avg(r.score_staff) AS avg_staff,
     avg(r.score_value_for_money) AS avg_value,
     count(r) AS review_count
WHERE review_count > 0
RETURN h.name AS hotel_name,
       h.star_rating AS star_rating,
       city.name AS city,
       country.name AS country,
       traveller_type,
       avg_overall, avg_cleanliness, avg_comfort, avg_facilities,
       avg_location, avg_staff, avg_value, review_count
ORDER BY h.name, traveller_type
LIMIT 1000
"""

hotel_data = conn_rag.execute_query(query)
print(f"Extracted {len(hotel_data)} hotel-traveller combinations")
if hotel_data:
    print(f"Sample: {json.dumps(hotel_data[0], indent=2)}")

Extracted 100 hotel-traveller combinations
Sample: {
  "hotel_name": "Aztec Heights",
  "star_rating": 5.0,
  "city": "Mexico City",
  "country": "Mexico",
  "traveller_type": "Business",
  "avg_overall": 8.624821002386632,
  "avg_cleanliness": 8.19832935560859,
  "avg_comfort": 8.611217183770881,
  "avg_facilities": 8.66825775656324,
  "avg_location": 9.26945107398567,
  "avg_staff": 8.811217183770879,
  "avg_value": 7.740334128878283,
  "review_count": 419
}


In [18]:
check_query = """
MATCH (h:Hotel)
OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)<-[:WROTE]-(t:Traveller)
WITH h, count(DISTINCT t.type) as traveller_types_with_reviews, count(r) as total_reviews
RETURN count(h) as total_hotels,
       sum(CASE WHEN total_reviews > 0 THEN 1 ELSE 0 END) as hotels_with_reviews,
       avg(traveller_types_with_reviews) as avg_traveller_types_per_hotel
"""

stats = conn_rag.execute_query(check_query)
print("DATABASE STATISTICS:")
print(json.dumps(stats[0], indent=2))

print("\nSample of hotel_data extracted:")
print(f"Total combinations: {len(hotel_data)}")
print(f"Unique hotels: {len(set(h['hotel_name'] for h in hotel_data))}")
print(f"Unique cities: {len(set(h['city'] for h in hotel_data))}")

traveller_dist = {}
for h in hotel_data:
    t = h.get('traveller_type')
    traveller_dist[t] = traveller_dist.get(t, 0) + 1

print(f"\nTraveller type distribution in extracted data:")
for t, count in sorted(traveller_dist.items()):
    print(f"  {t}: {count}")

DATABASE STATISTICS:
{
  "total_hotels": 25,
  "hotels_with_reviews": 25,
  "avg_traveller_types_per_hotel": 4.0
}

Sample of hotel_data extracted:
Total combinations: 100
Unique hotels: 25
Unique cities: 25

Traveller type distribution in extracted data:
  Business: 25
  Couple: 25
  Family: 25
  Solo: 25


### Step 2: Generate Natural Review Text

In [19]:
import random

def generate_review(hotel_info: Dict[str, Any]) -> str:
    traveller_type = hotel_info.get('traveller_type', 'solo')
    hotel_name = hotel_info['hotel_name']
    city = hotel_info['city']
    country = hotel_info['country']
    star = int(hotel_info.get('star_rating', 3))
    
    scores = {
        'overall': hotel_info.get('avg_overall', 7.0),
        'cleanliness': hotel_info.get('avg_cleanliness', 7.0),
        'comfort': hotel_info.get('avg_comfort', 7.0),
        'facilities': hotel_info.get('avg_facilities', 7.0),
        'location': hotel_info.get('avg_location', 7.0),
        'staff': hotel_info.get('avg_staff', 7.0),
        'value': hotel_info.get('avg_value', 7.0)
    }
    
    traveller_contexts = {
        'family': ['with my family', 'with the kids', 'as a family of four', 'family vacation'],
        'couple': ['with my partner', 'romantic getaway', 'anniversary trip', 'couples retreat'],
        'solo': ['solo trip', 'traveling alone', 'on my own', 'business trip'],
        'business': ['business trip', 'work travel', 'conference stay', 'corporate visit'],
        'group': ['with friends', 'group trip', 'with colleagues', 'friends vacation']
    }
    
    tone_templates = [
        'enthusiastic',
        'satisfied',
        'balanced',
        'critical_but_fair'
    ]
    
    tone = random.choice(tone_templates)
    context = random.choice(traveller_contexts.get(traveller_type, ['trip']))
    
    prompt = f"""Write a natural hotel review as if you're a real {traveller_type} traveler.

Hotel: {hotel_name} ({star}-star) in {city}, {country}
Trip context: {context}
Tone: {tone}

Aspect scores (0-10 scale):
- Overall: {scores['overall']:.1f}
- Cleanliness: {scores['cleanliness']:.1f}
- Comfort: {scores['comfort']:.1f}
- Facilities: {scores['facilities']:.1f}
- Location: {scores['location']:.1f}
- Staff: {scores['staff']:.1f}
- Value for money: {scores['value']:.1f}

RULES: 80-150 words in first person
1. Write
2. Sound human and conversational (not formal)
3. Mention 2-4 aspects naturally (don't list all)
4. Convert scores to natural expressions:
   - 9.0+: "amazing", "spotless", "fantastic", "excellent"
   - 8.0-8.9: "great", "very good", "really nice", "impressive"
   - 7.0-7.9: "good", "decent", "solid", "comfortable"
   - 6.0-6.9: "okay", "average", "acceptable", "could be better"
   - <6.0: "disappointing", "needs improvement", "not great"
5. Include traveller context naturally
6. Vary sentence structure and vocabulary
7. NO numeric ratings or formal language
8. Focus on experience, not data

Return ONLY the review text."""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8,
            max_tokens=250
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating review: {e}")
        return ""

test_review = generate_review(hotel_data[0])
print(f"Generated review ({len(test_review.split())} words):\n")
print(test_review)

Generated review (117 words):

I recently stayed at Aztec Heights during a business trip to Mexico City, and overall, it was a really nice experience. The location is fantastic, right in the heart of the city, making it easy to get to meetings and explore a bit in the evenings. The staff were friendly and attentive, which made a big difference during my stay. However, I did notice that the cleanliness could be improved—there were a few areas that felt a bit overlooked. The comfort level of the room was impressive, though; I slept well and felt rested for my busy days ahead. While the value for money could be better, I appreciated the overall vibe and would consider staying again.


### Step 3: Generate 100 Reviews for RAG

In [20]:
import time

# Check if reviews already exist
reviews_file = 'synthetic_reviews.json'

if os.path.exists(reviews_file):
    print(f"Loading existing reviews from {reviews_file}...")
    with open(reviews_file, 'r', encoding='utf-8') as f:
        generated_reviews = json.load(f)
    print(f"✓ Loaded {len(generated_reviews)} existing reviews")
    print(f"  Unique hotels: {len(set(r['hotel_name'] for r in generated_reviews))}")
    print(f"  File size: {os.path.getsize(reviews_file) / 1024:.1f} KB")
else:
    print(f"No existing reviews found. Generating new reviews...")
    
    def generate_reviews_batch(hotel_data_list, target_count=100, delay=0.3):
        reviews = []
        
        reviews_per_hotel = max(1, target_count // len(hotel_data_list))
        if reviews_per_hotel * len(hotel_data_list) < target_count:
            reviews_per_hotel += 1
        
        print(f"Generating {target_count} reviews from {len(hotel_data_list)} hotel-traveller combinations")
        print(f"Creating {reviews_per_hotel} reviews per combination\n")
        
        total_needed = target_count
        generated = 0
        
        while generated < total_needed:
            for hotel_info in hotel_data_list:
                if generated >= total_needed:
                    break
                
                try:
                    review_text = generate_review(hotel_info)
                    
                    if review_text:
                        review_entry = {
                            'hotel_name': hotel_info['hotel_name'],
                            'city': hotel_info['city'],
                            'country': hotel_info['country'],
                            'star_rating': hotel_info['star_rating'],
                            'traveller_type': hotel_info.get('traveller_type', 'solo'),
                            'review_text': review_text,
                            'metadata': {
                                'avg_overall': hotel_info.get('avg_overall'),
                                'avg_cleanliness': hotel_info.get('avg_cleanliness'),
                                'avg_comfort': hotel_info.get('avg_comfort'),
                                'avg_location': hotel_info.get('avg_location')
                            }
                        }
                        reviews.append(review_entry)
                        generated += 1
                    
                    if generated % 100 == 0:
                        print(f"\n{'='*70}")
                        print(f"Progress: {generated}/{total_needed} reviews generated")
                        print(f"{'='*70}")
                        
                        if len(reviews) >= 2:
                            print(f"\nSAMPLE: Review #{generated-1}")
                            print(f"Hotel: {reviews[-1]['hotel_name']} ({reviews[-1]['traveller_type']})")
                            print(f"Text: {reviews[-1]['review_text'][:200]}...")
                            
                            print(f"\nSAMPLE: Review #{generated-2}")
                            print(f"Hotel: {reviews[-2]['hotel_name']} ({reviews[-2]['traveller_type']})")
                            print(f"Text: {reviews[-2]['review_text'][:200]}...")
                        
                        word_counts = [len(r['review_text'].split()) for r in reviews]
                        print(f"\nQuality Check:")
                        print(f"  Avg word count: {sum(word_counts)/len(word_counts):.1f}")
                        print(f"  Unique hotels: {len(set(r['hotel_name'] for r in reviews))}")
                        print(f"{'='*70}\n")
                    
                    time.sleep(delay)
                    
                except Exception as e:
                    print(f"Error at review {generated}: {e}")
                    continue
        
        print(f"\nCompleted: {len(reviews)}/{total_needed} reviews generated successfully")
        return reviews
    
    generated_reviews = generate_reviews_batch(hotel_data, target_count=100, delay=0.3)
    
    # Save for next time
    with open(reviews_file, 'w', encoding='utf-8') as f:
        json.dump(generated_reviews, f, indent=2, ensure_ascii=False)
    print(f"\n✓ Saved {len(generated_reviews)} reviews to {reviews_file}")

print(f"\nTotal reviews available: {len(generated_reviews)}")
print(f"Sample review:\n{generated_reviews[0]['review_text']}")

Loading existing reviews from synthetic_reviews.json...
✓ Loaded 100 existing reviews
  Unique hotels: 25
  File size: 104.4 KB

Total reviews available: 100
Sample review:
I recently stayed at Aztec Heights during a business trip to Mexico City, and I have to say, I was quite impressed. The location is fantastic—right in the heart of the city and close to everything I needed for my meetings. The staff were really nice too; they went out of their way to help me with some last-minute arrangements. The room itself was comfortable, which was a relief after long days of work. I also appreciated the facilities; they had everything I needed to stay productive. Overall, it was a great experience, though I felt the value for money could be a touch better. But all in all, I’d definitely recommend it for anyone traveling on business!


### Quality Check: Review Diversity

In [21]:
print(f"Total reviews generated: {len(generated_reviews)}\n")

test_hotel = generated_reviews[0]['hotel_name']
test_type = generated_reviews[0]['traveller_type']

same_combo = [r for r in generated_reviews 
              if r['hotel_name'] == test_hotel and r['traveller_type'] == test_type]

print(f"{'='*70}")
print(f"DIVERSITY TEST: {test_hotel} ({test_type})")
print(f"Found {len(same_combo)} reviews for this combination")
print(f"{'='*70}\n")

for i, r in enumerate(same_combo[:4], 1):
    print(f"Review {i}:")
    print(r['review_text'])
    print(f"\n{'-'*70}\n")

word_counts = [len(r['review_text'].split()) for r in generated_reviews]
unique_starts = set(r['review_text'][:30] for r in generated_reviews)

print(f"\n{'='*70}")
print("OVERALL QUALITY METRICS")
print(f"{'='*70}")
print(f"Total reviews: {len(generated_reviews)}")
print(f"Unique hotels: {len(set(r['hotel_name'] for r in generated_reviews))}")
print(f"Unique starting phrases: {len(unique_starts)}/{len(generated_reviews)}")
print(f"\nWord count:")
print(f"  Min: {min(word_counts)}")
print(f"  Avg: {sum(word_counts)/len(word_counts):.1f}")
print(f"  Max: {max(word_counts)}")
print(f"\nTraveller type distribution:")
for ttype in set(r['traveller_type'] for r in generated_reviews):
    count = sum(1 for r in generated_reviews if r['traveller_type'] == ttype)
    print(f"  {ttype}: {count}")
print(f"{'='*70}")

Total reviews generated: 100

DIVERSITY TEST: Aztec Heights (Business)
Found 1 reviews for this combination

Review 1:
I recently stayed at Aztec Heights during a business trip to Mexico City, and I have to say, I was quite impressed. The location is fantastic—right in the heart of the city and close to everything I needed for my meetings. The staff were really nice too; they went out of their way to help me with some last-minute arrangements. The room itself was comfortable, which was a relief after long days of work. I also appreciated the facilities; they had everything I needed to stay productive. Overall, it was a great experience, though I felt the value for money could be a touch better. But all in all, I’d definitely recommend it for anyone traveling on business!

----------------------------------------------------------------------


OVERALL QUALITY METRICS
Total reviews: 100
Unique hotels: 25
Unique starting phrases: 72/100

Word count:
  Min: 98
  Avg: 119.3
  Max: 148

Tra

### Step 4: Save Reviews

In [22]:
reviews_file = 'synthetic_reviews.json'

with open(reviews_file, 'w', encoding='utf-8') as f:
    json.dump(generated_reviews, f, indent=2, ensure_ascii=False)

print(f"Saved {len(generated_reviews)} reviews to {reviews_file}")
print(f"File size: {os.path.getsize(reviews_file) / 1024:.1f} KB")

Saved 100 reviews to synthetic_reviews.json
File size: 104.4 KB


### Step 5: Verify Review Quality & Diversity

In [23]:
from collections import Counter

word_counts = [len(r['review_text'].split()) for r in generated_reviews]
traveller_types = [r['traveller_type'] for r in generated_reviews]
unique_hotels = len(set(r['hotel_name'] for r in generated_reviews))
unique_cities = len(set(r['city'] for r in generated_reviews))

print("DIVERSITY ANALYSIS")
print("=" * 60)
print(f"Total reviews: {len(generated_reviews)}")
print(f"Unique hotels: {unique_hotels}")
print(f"Unique cities: {unique_cities}")
print(f"\nWord count: min={min(word_counts)}, max={max(word_counts)}, avg={sum(word_counts)/len(word_counts):.1f}")
print(f"\nTraveller type distribution:")
for ttype, count in Counter(traveller_types).most_common():
    print(f"  {ttype}: {count} ({count/len(generated_reviews)*100:.1f}%)")

print(f"\n{'=' * 60}")
print("SAMPLE REVIEWS")
print("=" * 60)

sample_types = list(set(traveller_types))[:3]
for ttype in sample_types:
    sample = next((r for r in generated_reviews if r['traveller_type'] == ttype), None)
    if sample:
        print(f"\n[{ttype.upper()}] {sample['hotel_name']}, {sample['city']}")
        print(f"{sample['review_text']}")
        print("-" * 60)

DIVERSITY ANALYSIS
Total reviews: 100
Unique hotels: 25
Unique cities: 25

Word count: min=98, max=148, avg=119.3

Traveller type distribution:
  Business: 25 (25.0%)
  Couple: 25 (25.0%)
  Family: 25 (25.0%)
  Solo: 25 (25.0%)

SAMPLE REVIEWS

[FAMILY] Aztec Heights, Mexico City
We just returned from a family trip to Mexico City, and staying at Aztec Heights was a real highlight. The facilities were fantastic, offering something for everyone—from a lovely pool where the kids could splash around to a cozy lounge area where we could unwind after a long day of exploring. We also appreciated how clean everything was; it felt spotless and well-maintained, which is always a plus when traveling with little ones. The staff was very good, always friendly and ready to help us with our questions about the area. The location was decent, making it easy to access popular spots without too much hassle. Overall, it was a great experience, and we’d definitely consider coming back!
--------------------

### Step 6: Create Embeddings with TWO Models

In [24]:
import numpy as np
%pip install tf-keras
from sentence_transformers import SentenceTransformer

embedder_minilm = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embedder_mpnet = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

review_texts = [r['review_text'] for r in generated_reviews]

print("Creating embeddings with Model 1: all-MiniLM-L6-v2...")
embeddings_minilm = embedder_minilm.encode(review_texts, convert_to_numpy=True, show_progress_bar=True)

print("\nCreating embeddings with Model 2: all-mpnet-base-v2...")
embeddings_mpnet = embedder_mpnet.encode(review_texts, convert_to_numpy=True, show_progress_bar=True)

print(f"\nModel 1 (MiniLM): shape={embeddings_minilm.shape}, dim={embeddings_minilm.shape[1]}")
print(f"Model 2 (MPNet): shape={embeddings_mpnet.shape}, dim={embeddings_mpnet.shape[1]}")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Creating embeddings with Model 1: all-MiniLM-L6-v2...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]


Creating embeddings with Model 2: all-mpnet-base-v2...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]


Model 1 (MiniLM): shape=(100, 384), dim=384
Model 2 (MPNet): shape=(100, 768), dim=768


### Step 7: Store in Neo4j Vector Index

In [25]:
for i, review in enumerate(generated_reviews):
    review['review_id'] = f"review_{i}"
    review['embedding_minilm'] = embeddings_minilm[i].tolist()
    review['embedding_mpnet'] = embeddings_mpnet[i].tolist()

print(f"Added embeddings to {len(generated_reviews)} review objects")
print(f"Sample review keys: {list(generated_reviews[0].keys())}")

Added embeddings to 100 review objects
Sample review keys: ['hotel_name', 'city', 'country', 'star_rating', 'traveller_type', 'review_text', 'metadata', 'review_id', 'embedding_minilm', 'embedding_mpnet']


In [26]:
create_nodes_query = """
UNWIND $reviews AS review
CREATE (sr:SyntheticReview {
    review_id: review.review_id,
    hotel_name: review.hotel_name,
    city: review.city,
    country: review.country,
    star_rating: review.star_rating,
    traveller_type: review.traveller_type,
    review_text: review.review_text,
    embedding_minilm: review.embedding_minilm,
    embedding_mpnet: review.embedding_mpnet
})
"""

print("Creating SyntheticReview nodes in Neo4j...")
conn_rag.execute_query(create_nodes_query, {'reviews': generated_reviews})

count_query = "MATCH (sr:SyntheticReview) RETURN count(sr) as total"
result = conn_rag.execute_query(count_query)
print(f"Created {result[0]['total']} SyntheticReview nodes")

Creating SyntheticReview nodes in Neo4j...
Created 100 SyntheticReview nodes


### Step 8: Create Vector Indices

In [27]:
index_minilm_query = """
CREATE VECTOR INDEX review_minilm_index IF NOT EXISTS
FOR (sr:SyntheticReview)
ON sr.embedding_minilm
OPTIONS {indexConfig: {
 `vector.dimensions`: 384,
 `vector.similarity_function`: 'cosine'
}}
"""

index_mpnet_query = """
CREATE VECTOR INDEX review_mpnet_index IF NOT EXISTS
FOR (sr:SyntheticReview)
ON sr.embedding_mpnet
OPTIONS {indexConfig: {
 `vector.dimensions`: 768,
 `vector.similarity_function`: 'cosine'
}}
"""

print("Creating vector index for Model 1 (MiniLM - 384 dim)...")
conn_rag.execute_query(index_minilm_query)

print("Creating vector index for Model 2 (MPNet - 768 dim)...")
conn_rag.execute_query(index_mpnet_query)

print("\nVector indices created successfully!")

Creating vector index for Model 1 (MiniLM - 384 dim)...
Creating vector index for Model 2 (MPNet - 768 dim)...

Vector indices created successfully!


### Step 9: Semantic Search Functions (Both Models)

In [28]:
def semantic_search_minilm(query: str, top_k: int = 5, threshold: float = 0.65):
    """
    Semantic search with MiniLM embeddings and similarity threshold.
    
    Args:
        query: Search query
        top_k: Maximum number of results
        threshold: Minimum similarity score (0-1). Results below this are filtered out.
    
    Returns:
        List of results with similarity >= threshold
    """
    query_embedding = embedder_minilm.encode([query], convert_to_numpy=True)[0].tolist()
    
    search_query = """
    CALL db.index.vector.queryNodes('review_minilm_index', $top_k, $query_embedding)
    YIELD node, score
    RETURN node.review_id AS review_id,
           node.hotel_name AS hotel_name,
           node.city AS city,
           node.country AS country,
           node.traveller_type AS traveller_type,
           node.review_text AS review_text,
           score
    """
    
    results = conn_rag.execute_query(search_query, {
        'query_embedding': query_embedding,
        'top_k': top_k
    })
    
    # Filter by similarity threshold
    filtered_results = [r for r in results if r['score'] >= threshold]
    
    return filtered_results

def semantic_search_mpnet(query: str, top_k: int = 5, threshold: float = 0.65):
    """
    Semantic search with MPNet embeddings and similarity threshold.
    
    Args:
        query: Search query
        top_k: Maximum number of results
        threshold: Minimum similarity score (0-1). Results below this are filtered out.
    
    Returns:
        List of results with similarity >= threshold
    """
    query_embedding = embedder_mpnet.encode([query], convert_to_numpy=True)[0].tolist()
    
    search_query = """
    CALL db.index.vector.queryNodes('review_mpnet_index', $top_k, $query_embedding)
    YIELD node, score
    RETURN node.review_id AS review_id,
           node.hotel_name AS hotel_name,
           node.city AS city,
           node.country AS country,
           node.traveller_type AS traveller_type,
           node.review_text AS review_text,
           score
    """
    
    results = conn_rag.execute_query(search_query, {
        'query_embedding': query_embedding,
        'top_k': top_k
    })
    
    # Filter by similarity threshold
    filtered_results = [r for r in results if r['score'] >= threshold]
    
    return filtered_results

print("Semantic search functions with threshold filtering defined")
print("Default threshold: 0.65 (moderate relevance)")
print("Recommended thresholds:")
print("  - 0.80+: Very high similarity")
print("  - 0.70-0.80: Good match")
print("  - 0.60-0.70: Moderate match")
print("  - < 0.60: Low relevance (filtered out)")

Semantic search functions with threshold filtering defined
Default threshold: 0.65 (moderate relevance)
Recommended thresholds:
  - 0.80+: Very high similarity
  - 0.70-0.80: Good match
  - 0.60-0.70: Moderate match
  - < 0.60: Low relevance (filtered out)


### Step 10: Compare Both Embedding Models

In [29]:
import time

test_queries = [
    "Looking for a clean hotel with great staff for my family",
    "Romantic hotel with amazing views",
    "Budget-friendly hotel with good value for money",
]

print("EMBEDDING MODEL COMPARISON")
print("=" * 80)
print("Model 1: all-MiniLM-L6-v2 (384 dimensions)")
print("Model 2: all-mpnet-base-v2 (768 dimensions)")
print("=" * 80)

comparison_results = []

for test_query in test_queries:
    print(f"\n\nQuery: \"{test_query}\"")
    print("-" * 80)
    
    start_time = time.time()
    results_minilm = semantic_search_minilm(test_query, top_k=3)
    time_minilm = time.time() - start_time
    
    start_time = time.time()
    results_mpnet = semantic_search_mpnet(test_query, top_k=3)
    time_mpnet = time.time() - start_time
    
    print(f"\nModel 1 (MiniLM): {len(results_minilm)} results in {time_minilm:.3f}s")
    if results_minilm:
        top = results_minilm[0]
        print(f"  Top match: {top['hotel_name']} ({top['city']}) - Score: {top['score']:.4f}")
        print(f"  Review: {top['review_text'][:100]}...")
    
    print(f"\nModel 2 (MPNet): {len(results_mpnet)} results in {time_mpnet:.3f}s")
    if results_mpnet:
        top = results_mpnet[0]
        print(f"  Top match: {top['hotel_name']} ({top['city']}) - Score: {top['score']:.4f}")
        print(f"  Review: {top['review_text'][:100]}...")
    
    comparison_results.append({
        'query': test_query,
        'minilm_time': time_minilm,
        'mpnet_time': time_mpnet,
        'minilm_top_score': results_minilm[0]['score'] if results_minilm else 0,
        'mpnet_top_score': results_mpnet[0]['score'] if results_mpnet else 0
    })

print(f"\n\n{'=' * 80}")
print("SUMMARY")
print("=" * 80)
avg_time_minilm = sum(r['minilm_time'] for r in comparison_results) / len(comparison_results)
avg_time_mpnet = sum(r['mpnet_time'] for r in comparison_results) / len(comparison_results)
avg_score_minilm = sum(r['minilm_top_score'] for r in comparison_results) / len(comparison_results)
avg_score_mpnet = sum(r['mpnet_top_score'] for r in comparison_results) / len(comparison_results)

print(f"Average search time - MiniLM: {avg_time_minilm:.3f}s | MPNet: {avg_time_mpnet:.3f}s")
print(f"Average top score - MiniLM: {avg_score_minilm:.4f} | MPNet: {avg_score_mpnet:.4f}")
print(f"\nSpeed winner: {'MiniLM' if avg_time_minilm < avg_time_mpnet else 'MPNet'}")
print(f"Quality winner: {'MiniLM' if avg_score_minilm > avg_score_mpnet else 'MPNet'} (higher score = better match)")

EMBEDDING MODEL COMPARISON
Model 1: all-MiniLM-L6-v2 (384 dimensions)
Model 2: all-mpnet-base-v2 (768 dimensions)


Query: "Looking for a clean hotel with great staff for my family"
--------------------------------------------------------------------------------

Model 1 (MiniLM): 3 results in 1.283s
  Top match: The Golden Oasis (Dubai) - Score: 0.7637
  Review: I recently stayed at The Golden Oasis during a business trip to Dubai, and I was quite impressed. Th...

Model 2 (MPNet): 3 results in 0.148s
  Top match: The Kiwi Grand (Wellington) - Score: 0.7875
  Review: We just wrapped up an incredible stay at The Kiwi Grand in Wellington, and I can’t recommend it enou...


Query: "Romantic hotel with amazing views"
--------------------------------------------------------------------------------

Model 1 (MiniLM): 3 results in 0.089s
  Top match: Nile Grandeur (Cairo) - Score: 0.7862
  Review: We recently stayed at the Nile Grandeur in Cairo, and it was such a delightful experience! The 

# Part 3: LLM Layer

Combines Knowledge Graph results (baseline + embeddings) and generates natural language responses using GPT models.

In [30]:
# Ensure we have an active Neo4j connection for Part 3
try:
    # Test if connection is still active
    conn.execute_query("RETURN 1")
    print("Using existing Neo4j connection")
except:
    # Create new connection if previous one was closed
    conn = Neo4jConnection()
    print("Created new Neo4j connection for Part 3")

Created new Neo4j connection for Part 3


## 3.a Combine KG Results (Baseline + Embeddings)

Smart merging and deduplication of Cypher query results and RAG embedding results.

In [31]:
from collections import defaultdict
from typing import Tuple

def merge_and_rank_results(
    cypher_output: List[Dict],
    embedding_output: Optional[List[Dict]],
    intent: str
) -> Dict[str, Any]:
    """
    Intelligently merge KG and embedding results.
    - Deduplicates hotels
    - Ranks by relevance
    - Enriches KG data with review insights
    """
    merged = {
        'primary_results': [],
        'supporting_reviews': [],
        'metadata': {
            'cypher_count': len(cypher_output) if cypher_output else 0,
            'embedding_count': len(embedding_output) if embedding_output else 0
        }
    }
    
    if not cypher_output and not embedding_output:
        merged['metadata']['has_results'] = False
        return merged
    
    merged['metadata']['has_results'] = True
    
    # Use KG results as primary source (structured data)
    if cypher_output:
        merged['primary_results'] = cypher_output
    
    # Add embedding results as supporting evidence
    if embedding_output:
        if intent in ["RECOMMEND_HOTEL", "DESCRIBE_HOTEL", "COMPARE_HOTELS"]:
            # Group reviews by hotel for context enrichment
            reviews_by_hotel = defaultdict(list)
            for review in embedding_output:
                hotel_name = review.get('hotel_name', '')
                reviews_by_hotel[hotel_name].append(review)
            
            merged['supporting_reviews'] = dict(reviews_by_hotel)
        else:
            # For LIST intent, just include unique hotels not in KG results
            kg_hotels = {r.get('hotel_name', '') for r in cypher_output} if cypher_output else set()
            unique_embedding = [
                r for r in embedding_output 
                if r.get('hotel_name', '') not in kg_hotels
            ]
            merged['supporting_reviews'] = unique_embedding[:5]
    
    return merged

print("merge_and_rank_results function defined")

merge_and_rank_results function defined


## 3.b Structured Prompts (Context + Persona + Task)

Intent-specific context builders and prompt engineering with clear persona and task definitions.

### Prompt Engine with Intent-Specific Personas

In [32]:
class PromptEngine:
    """Generates optimized prompts for each intent with few-shot examples."""
    
    @staticmethod
    def get_prompts(intent: str, query: str, context: str) -> Tuple[str, str]:
        """Generate system and user prompts."""
        
        base_system = """You are an expert hotel assistant with access to a comprehensive hotel knowledge graph.

CRITICAL RULES:
1. Answer EXCLUSIVELY using the provided context data
2. NEVER use external knowledge or assumptions
3. If data is insufficient, explicitly state what's missing
4. Be precise, accurate, and cite specific numbers from the context
5. Maintain a professional yet friendly tone"""
        
        if intent == "LIST_HOTELS":
            system = base_system + """

TASK: Present hotels as a clear, scannable list
FORMAT: Numbered list with key details (name, location, rating)
TONE: Concise and helpful"""
            
            user = f"""CONTEXT:
{context}

USER QUERY: "{query}"

Provide a clear numbered list of hotels:"""

        elif intent == "RECOMMEND_HOTEL":
            system = base_system + """

TASK: Recommend hotels and explain WHY
FORMAT: Top 2-3 recommendations with reasoning
TONE: Persuasive but honest
INCLUDE: Specific scores, review counts, and guest feedback quotes"""
            
            user = f"""CONTEXT:
{context}

USER QUERY: "{query}"

Provide recommendations with clear reasoning:"""

        elif intent == "DESCRIBE_HOTEL":
            system = base_system + """

TASK: Provide comprehensive hotel description
FORMAT: Structured overview covering all available aspects
TONE: Informative and balanced
INCLUDE: Base ratings, review scores, and guest experiences"""
            
            user = f"""CONTEXT:
{context}

USER QUERY: "{query}"

Provide a detailed, well-structured description:"""

        elif intent == "COMPARE_HOTELS":
            system = base_system + """

TASK: Compare hotels objectively using base ratings
FORMAT: Side-by-side comparison highlighting differences
TONE: Analytical and balanced
INCLUDE: Specific rating differences, clear winner per category
IMPORTANT: Use the BASE RATINGS for comparison (not review scores)"""
            
            user = f"""CONTEXT:
{context}

USER QUERY: "{query}"

Provide a structured comparison:"""

        elif intent == "CHECK_VISA":
            system = base_system + """

TASK: Provide visa requirement information
FORMAT: Start with clear YES/NO, then details
TONE: Factual and authoritative
INCLUDE: Visa type if applicable"""
            
            user = f"""CONTEXT:
{context}

USER QUERY: "{query}"

Provide clear visa information:"""
        
        else:
            system = base_system
            user = f"Context:\\n{context}\\n\\nQuestion: {query}\\n\\nAnswer:"
        
        return system, user

print("PromptEngine class defined")

PromptEngine class defined


### Main LLM Layer Function

In [33]:
def llm_layer(
    user_query: str,
    intent: str,
    cypher_output: List[Dict],
    embedding_output: Optional[List[Dict]] = None,
    model: str = "gpt-4o-mini",
    temperature: float = 0.0,
    max_tokens: int = 1000
) -> Dict[str, Any]:
    """
    Production-grade LLM layer with:
    - Smart result merging
    - Intent-optimized prompts
    - Few-shot examples
    - Output validation
    
    Args:
        user_query: Original user question
        intent: Classified intent
        cypher_output: Results from baseline Cypher queries
        embedding_output: Optional RAG results
        model: GPT model (gpt-4o-mini, gpt-4o, gpt-4-turbo)
        temperature: 0.0 for deterministic, higher for creative
        max_tokens: Max response length
    
    Returns:
        Complete response with metadata and quality metrics
    """
    
    # Step 1: Merge and deduplicate results
    merged_data = merge_and_rank_results(cypher_output, embedding_output, intent)
    
    # Step 2: Build optimized context
    context = ContextBuilder.build(intent, merged_data)
    
    # Step 3: Generate intent-specific prompts
    system_prompt, user_prompt = PromptEngine.get_prompts(intent, user_query, context)
    
    # Step 4: Call LLM with error handling
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        answer = response.choices[0].message.content
        tokens_used = response.usage.total_tokens
        finish_reason = response.choices[0].finish_reason
        
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'model': model,
            'intent': intent
        }
    
    # Step 5: Return comprehensive result
    return {
        'success': True,
        'model': model,
        'intent': intent,
        'response': answer,
        'metadata': {
            'query': user_query,
            'cypher_results_count': merged_data['metadata']['cypher_count'],
            'embedding_results_count': merged_data['metadata']['embedding_count'],
            'has_results': merged_data['metadata']['has_results'],
            'tokens_used': tokens_used,
            'finish_reason': finish_reason,
            'temperature': temperature
        },
        'context_used': context[:500] + "..." if len(context) > 500 else context,
        'full_context': context
    }

print("llm_layer function defined")

llm_layer function defined


In [34]:
from typing import Dict, Any
import json

class ContextBuilder:
    """Builds optimized context for each intent type."""

    @staticmethod
    def build(intent: str, merged_data: Dict[str, Any]) -> str:
        """Route to intent-specific builder."""
        builders = {
            'LIST_HOTELS': ContextBuilder._build_list,
            'RECOMMEND_HOTEL': ContextBuilder._build_recommend,
            'DESCRIBE_HOTEL': ContextBuilder._build_describe,
            'COMPARE_HOTELS': ContextBuilder._build_compare,
            'CHECK_VISA': ContextBuilder._build_visa
        }

        builder = builders.get(intent, ContextBuilder._build_generic)
        return builder(merged_data)

    @staticmethod
    def _get_reviews_for_hotel(data: Dict[str, Any], hotel_name: str):
        """
        supporting_reviews might be:
        - dict: {hotel_name: [review_dicts]}
        - list: [review_dicts] (no hotel grouping)
        """
        sup = data.get("supporting_reviews")
        if not sup:
            return []

        if isinstance(sup, dict):
            return sup.get(hotel_name, []) or []

        if isinstance(sup, list):
            # If list of review dicts contains hotel_name field, filter it
            out = []
            for r in sup:
                if isinstance(r, dict) and r.get("hotel_name") == hotel_name:
                    out.append(r)
            return out

        return []

    @staticmethod
    def _build_list(data: Dict[str, Any]) -> str:
        if not data.get('metadata', {}).get('has_results'):
            return "No hotels found matching the criteria."

        context = "=== AVAILABLE HOTELS ===\n\n"
        for idx, hotel in enumerate(data.get('primary_results', []), 1):
            context += f"Hotel #{idx}\n"
            context += f"  Name: {hotel.get('hotel_name', 'N/A')}\n"
            context += f"  Location: {hotel.get('city_name', 'N/A')}, {hotel.get('country_name', 'N/A')}\n"
            context += f"  Star Rating: {hotel.get('star_rating', 'N/A')}/5\n"
            context += "\n"

        # Additional options from reviews (works whether supporting_reviews is list or dict)
        sup = data.get("supporting_reviews")
        extra = []
        if isinstance(sup, list):
            extra = sup[:5]
        elif isinstance(sup, dict):
            # pick 1 review per hotel up to 5 hotels
            for hname, reviews in list(sup.items())[:5]:
                if reviews:
                    extra.append(reviews[0] if isinstance(reviews, list) else {"hotel_name": hname})

        if extra:
            context += "=== ADDITIONAL OPTIONS FROM REVIEWS ===\n"
            for review in extra:
                context += f"  • {review.get('hotel_name', 'N/A')} in {review.get('city', review.get('city_name', 'N/A'))}\n"

        return context

    @staticmethod
    def _build_recommend(data: Dict[str, Any]) -> str:
        if not data.get('metadata', {}).get('has_results'):
            return "No recommendations available for this query."

        context = "=== HOTEL RECOMMENDATIONS ===\n\n"

        for idx, hotel in enumerate(data.get('primary_results', [])[:5], 1):
            hotel_name = hotel.get('hotel_name', 'Unknown')

            context += f"OPTION {idx}: {hotel_name}\n"
            context += f"Location: {hotel.get('city_name', 'N/A')}, {hotel.get('country_name', 'N/A')}\n\n"

            context += "SCORES:\n"
            if hotel.get('overall_review_score') is not None:
                context += f"  Overall: {float(hotel['overall_review_score']):.1f}/10\n"

            for aspect in ['cleanliness', 'comfort', 'facilities', 'location', 'staff', 'value_for_money']:
                key = f'{aspect}_review'
                if hotel.get(key) is not None:
                    context += f"  {aspect.replace('_', ' ').title()}: {float(hotel[key]):.1f}/10\n"

            if hotel.get('composite_aspect_score') is not None:
                context += f"  Composite Score: {float(hotel['composite_aspect_score']):.1f}/10\n"

            if hotel.get('review_count') is not None:
                context += f"  Based on: {hotel['review_count']} reviews\n"

            reviews = ContextBuilder._get_reviews_for_hotel(data, hotel_name)
            if reviews:
                context += "\nRECENT GUEST FEEDBACK:\n"
                for review in reviews[:2]:
                    traveller = review.get('traveller_type', 'Guest')
                    text = (review.get('review_text', '') or '')[:200]
                    context += f'  [{traveller}] "{text}..."\n'

            context += "\n" + "=" * 50 + "\n\n"

        return context

    @staticmethod
    def _build_describe(data: Dict[str, Any]) -> str:
        if not data.get('metadata', {}).get('has_results'):
            return "Hotel information not found."

        hotel = (data.get('primary_results') or [{}])[0]
        hotel_name = hotel.get('hotel_name', 'Unknown Hotel')

        context = f"=== {hotel_name.upper()} ===\n\n"
        context += f"Location: {hotel.get('city_name', 'N/A')}, {hotel.get('country_name', 'N/A')}\n\n"

        context += "HOTEL BASE STANDARDS:\n"
        for aspect in ['cleanliness', 'comfort', 'facilities', 'location', 'staff', 'value_for_money']:
            base_key = f'{aspect}_base'
            if hotel.get(base_key) is not None:
                context += f"  {aspect.replace('_', ' ').title()}: {hotel[base_key]}/10\n"

        context += "\nGUEST REVIEW SCORES:\n"
        for aspect in ['cleanliness', 'comfort', 'facilities', 'location', 'staff', 'value_for_money']:
            review_key = f'{aspect}_review'
            if hotel.get(review_key) is not None:
                context += f"  {aspect.replace('_', ' ').title()}: {float(hotel[review_key]):.1f}/10\n"

        if hotel.get('review_count') is not None:
            context += f"\nTotal Reviews: {hotel['review_count']}\n"

        reviews = ContextBuilder._get_reviews_for_hotel(data, hotel_name)
        if reviews:
            context += "\n=== GUEST EXPERIENCES ===\n\n"
            for idx, review in enumerate(reviews[:5], 1):
                traveller = review.get('traveller_type', 'Guest')
                text = review.get('review_text', '') or ''
                context += f"Review {idx} [{traveller}]:\n\"{text}\"\n\n"

        return context

    @staticmethod
    def _build_compare(data: Dict[str, Any]) -> str:
        if not data.get('metadata', {}).get('has_results'):
            return "Comparison data not available."

        comp = (data.get('primary_results') or [{}])[0]

        h1_name = comp.get('hotel1_name', 'Hotel 1')
        h2_name = comp.get('hotel2_name', 'Hotel 2')

        context = "=== HOTEL COMPARISON ===\n\n"
        context += f"HOTEL A: {h1_name}\n"
        context += f"Location: {comp.get('hotel1_city', 'N/A')}, {comp.get('hotel1_country', 'N/A')}\n\n"
        context += f"HOTEL B: {h2_name}\n"
        context += f"Location: {comp.get('hotel2_city', 'N/A')}, {comp.get('hotel2_country', 'N/A')}\n\n"

        context += "=== RATING COMPARISON (Base Standards) ===\n\n"
        context += f"{'Aspect':<20} | {h1_name[:15]:<15} | {h2_name[:15]:<15} | Difference\n"
        context += "-" * 75 + "\n"

        aspects = ['cleanliness', 'comfort', 'facilities', 'location', 'staff', 'value_for_money']
        for aspect in aspects:
            h1_key = f'hotel1_{aspect}_base'
            h2_key = f'hotel2_{aspect}_base'
            if comp.get(h1_key) is not None and comp.get(h2_key) is not None:
                h1_val = float(comp[h1_key])
                h2_val = float(comp[h2_key])
                diff = h1_val - h2_val
                diff_str = f"+{diff:.1f}" if diff > 0 else f"{diff:.1f}"
                aspect_name = aspect.replace('_', ' ').title()
                context += f"{aspect_name:<20} | {h1_val:>15.1f} | {h2_val:>15.1f} | {diff_str:>10}\n"

        # Add reviews if available
        sup = data.get("supporting_reviews")
        if sup:
            context += "\n=== GUEST REVIEWS ===\n\n"

            r1 = ContextBuilder._get_reviews_for_hotel(data, h1_name)
            if r1:
                context += f"--- {h1_name} ---\n"
                for review in r1[:2]:
                    context += f"[{review.get('traveller_type', 'Guest')}] {(review.get('review_text','') or '')[:150]}...\n\n"

            r2 = ContextBuilder._get_reviews_for_hotel(data, h2_name)
            if r2:
                context += f"--- {h2_name} ---\n"
                for review in r2[:2]:
                    context += f"[{review.get('traveller_type', 'Guest')}] {(review.get('review_text','') or '')[:150]}...\n\n"

        return context

    @staticmethod
    def _build_visa(data: Dict[str, Any]) -> str:
        if not data.get('metadata', {}).get('has_results'):
            return "Visa information not available."

        visa = (data.get('primary_results') or [{}])[0]

        context = "=== VISA REQUIREMENT ===\n\n"
        context += f"From Country: {visa.get('from_country', 'Unknown')}\n"
        context += f"To Country: {visa.get('to_country', 'Unknown')}\n"
        context += f"Visa Required: {'YES' if visa.get('visa_required') else 'NO'}\n"

        if visa.get('visa_type'):
            context += f"Visa Type: {visa['visa_type']}\n"

        return context

    @staticmethod
    def _build_generic(data: Dict[str, Any]) -> str:
        return json.dumps(data.get('primary_results', []), indent=2, ensure_ascii=False)


print("ContextBuilder class defined")


ContextBuilder class defined


## 3.c Model Comparison (Quantitative & Qualitative)

Compare at least 3 GPT models: gpt-4o-mini, gpt-4o, gpt-4-turbo

### Test Example: Single Model

In [35]:
# Example: Test with RECOMMEND_HOTEL intent
test_query = "Recommend hotels in Cairo with good cleanliness"
test_intent = "RECOMMEND_HOTEL"

# Get Cypher results
entities = extract_entities(test_query, test_intent)
cypher_results = select_and_execute_query(conn, test_intent, entities)

# Get embedding results (if needed)
embedding_results = semantic_search_mpnet(test_query, top_k=3)

# Generate LLM response
result = llm_layer(
    user_query=test_query,
    intent=test_intent,
    cypher_output=cypher_results,
    embedding_output=embedding_results,
    model="gpt-4o-mini"
)

print("=" * 80)
print("QUERY:", test_query)
print("=" * 80)
print("\\nLLM RESPONSE:")
print(result['response'])
print("\\n" + "=" * 80)
print("METADATA:")
print(f"  Cypher results: {result['metadata']['cypher_results_count']}")
print(f"  Embedding results: {result['metadata']['embedding_results_count']}")
print(f"  Tokens used: {result['metadata']['tokens_used']}")
print(f"  Model: {result['model']}")
print("=" * 80)

QUERY: Recommend hotels in Cairo with good cleanliness
\nLLM RESPONSE:
Based on your request for hotels in Cairo with good cleanliness, I recommend the following option:

### 1. Nile Grandeur
- **Cleanliness Score:** 8.9/10
- **Composite Score:** 8.9/10
- **Review Count:** 2104 reviews

**Reasoning:**
The Nile Grandeur stands out with an impressive cleanliness score of 8.9, indicating a high standard of hygiene and maintenance. Recent guest feedback highlights this aspect, with one family stating, "the cleanliness was impressive—our room felt spotless and welcoming." Additionally, the hotel's location by the Nile and proximity to major attractions enhances its appeal, making it a great choice for both families and couples. The positive experiences shared by guests further reinforce its reputation as a clean and enjoyable place to stay.

If you have any other specific preferences or need more options, please let me know!
\n================================================================

In [36]:
def compare_models(
    user_query: str,
    intent: str,
    cypher_output: List[Dict],
    embedding_output: Optional[List[Dict]] = None,
    models: List[str] = ["gpt-4o-mini", "gpt-4o", "gpt-4-turbo"]
) -> Dict[str, Any]:
    """Compare multiple models on the same query."""
    
    results = {}
    
    print(f"\\nComparing {len(models)} models on query: '{user_query}'")
    print("=" * 80)
    
    for model in models:
        print(f"\\nTesting {model}...")
        result = llm_layer(
            user_query=user_query,
            intent=intent,
            cypher_output=cypher_output,
            embedding_output=embedding_output,
            model=model
        )
        results[model] = result
        
        if result['success']:
            print(f"✓ Generated {len(result['response'])} chars, {result['metadata']['tokens_used']} tokens")
        else:
            print(f"✗ Error: {result['error']}")
    
    return {
        'query': user_query,
        'intent': intent,
        'model_results': results,
        'comparison': {
            'tokens_used': {m: results[m]['metadata']['tokens_used'] for m in models if results[m]['success']},
            'response_lengths': {m: len(results[m]['response']) for m in models if results[m]['success']}
        }
    }

print("compare_models function defined")

compare_models function defined


### Test: 3-Model Comparison (Quantitative + Qualitative)

In [37]:
# Test query for model comparison
comparison_query = "Compare The Azure Tower and Nile Grandeur"
comparison_intent = "COMPARE_HOTELS"

# Get results
entities_comp = extract_entities(comparison_query, comparison_intent)
cypher_comp = select_and_execute_query(conn, comparison_intent, entities_comp)
embedding_comp = semantic_search_mpnet(comparison_query, top_k=5, threshold=0.65)

# Compare 3 models
comparison_results = compare_models(
    user_query=comparison_query,
    intent=comparison_intent,
    cypher_output=cypher_comp,
    embedding_output=embedding_comp,
    models=["gpt-4o-mini", "gpt-4o", "gpt-4-turbo"]
)

# Display quantitative comparison
print("\\n" + "="*80)
print("QUANTITATIVE COMPARISON")
print("="*80)
print("\\nTokens Used:")
for model, tokens in comparison_results['comparison']['tokens_used'].items():
    print(f"  {model}: {tokens} tokens")

print("\\nResponse Lengths (characters):")
for model, length in comparison_results['comparison']['response_lengths'].items():
    print(f"  {model}: {length} chars")

# Display qualitative comparison (sample responses)
print("\\n" + "="*80)
print("QUALITATIVE COMPARISON")
print("="*80)

for model, result in comparison_results['model_results'].items():
    if result['success']:
        print(f"\\n{'='*80}")
        print(f"MODEL: {model}")
        print("="*80)
        print(result['response'])
        print()

\nComparing 3 models on query: 'Compare The Azure Tower and Nile Grandeur'
\nTesting gpt-4o-mini...
✓ Generated 1301 chars, 733 tokens
\nTesting gpt-4o...
✓ Generated 1562 chars, 790 tokens
\nTesting gpt-4-turbo...
✓ Generated 1637 chars, 891 tokens
\n================================================================================
QUANTITATIVE COMPARISON
\nTokens Used:
  gpt-4o-mini: 733 tokens
  gpt-4o: 790 tokens
  gpt-4-turbo: 891 tokens
\nResponse Lengths (characters):
  gpt-4o-mini: 1301 chars
  gpt-4o: 1562 chars
  gpt-4-turbo: 1637 chars
\n================================================================================
QUALITATIVE COMPARISON
\n================================================================================
MODEL: gpt-4o-mini
Here is a structured comparison of The Azure Tower and Nile Grandeur based on their base ratings:

| Aspect               | The Azure Tower | Nile Grandeur   | Difference         | Winner                |
|----------------------|------------